# Project Sprint 4:  

## Load Data

In [1]:
!pip install jupyter_bokeh
!pip install panel
!pip install plotly
!pip install kagglehub


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import kagglehub
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from kagglehub import KaggleDatasetAdapter
import plotly.express as px
import panel as pn
import panel.widgets as pnw
pn.extension("plotly", # for plotly
             "tabulator", # for displaying dataframes
            #  sizing_mode="stretch_width" # for folium
             )

# Download the latest dataset from kaggle
downloaded_path = kagglehub.dataset_download("thedevastator/spotify-tracks-genre-dataset")

# Load the dataset using dataset_load
songs = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "thedevastator/spotify-tracks-genre-dataset",
  "train.csv",
)

songs.dropna()
songs.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


## Transform

In [3]:
# outline of audio features and genres
audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
genres = songs['track_genre'].unique().tolist()

# Pair audio features and popularity column and genres
pop_v_features = audio_features + ['popularity'] + ['track_genre']
songs_pvf = songs[pop_v_features]

# handle missing values w/ a simple dropna statement
songs_pvf = songs_pvf.dropna()

# Define popularity bucket boundaries and labels
popularity_bins = [0, 20, 40, 60, 80, 100]
popularity_labels = ['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High']

# Create the popularity_bucket column
songs_pvf['popularity_bucket'] = pd.cut(songs_pvf['popularity'], bins=popularity_bins, labels=popularity_labels, include_lowest=True)

display(songs_pvf.head())
songs_pvf.info()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,track_genre,popularity_bucket
0,0.676,0.4610,-6.746,0.1430,0.0322,0.000001,0.3580,0.715,87.917,73,acoustic,Medium-High
1,0.420,0.1660,-17.235,0.0763,0.9240,0.000006,0.1010,0.267,77.489,55,acoustic,Medium
2,0.438,0.3590,-9.734,0.0557,0.2100,0.000000,0.1170,0.120,76.332,57,acoustic,Medium
3,0.266,0.0596,-18.515,0.0363,0.9050,0.000071,0.1320,0.143,181.740,71,acoustic,Medium-High
4,0.618,0.4430,-9.681,0.0526,0.4690,0.000000,0.0829,0.167,119.949,82,acoustic,High


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   danceability       114000 non-null  float64 
 1   energy             114000 non-null  float64 
 2   loudness           114000 non-null  float64 
 3   speechiness        114000 non-null  float64 
 4   acousticness       114000 non-null  float64 
 5   instrumentalness   114000 non-null  float64 
 6   liveness           114000 non-null  float64 
 7   valence            114000 non-null  float64 
 8   tempo              114000 non-null  float64 
 9   popularity         114000 non-null  int64   
 10  track_genre        114000 non-null  object  
 11  popularity_bucket  114000 non-null  category
dtypes: category(1), float64(9), int64(1), object(1)
memory usage: 9.7+ MB


In [5]:
# Create table summarizing means of each audio feature for each genre
genre_summary = songs_pvf.groupby('track_genre')[audio_features].mean().reset_index()
display(genre_summary.head())

,track_genre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,acoustic,0.549593,0.435368,-9.447843,0.043247,0.566816,0.038336,0.153244,0.424023,119.010624
1,afrobeat,0.669580,0.702812,-7.789353,0.086579,0.270860,0.253483,0.184596,0.698619,119.213337
2,alt-rock,0.534493,0.754173,-6.191489,0.055071,0.122162,0.054097,0.210249,0.518260,124.634404
3,alternative,0.559927,0.720030,-6.078777,0.070101,0.147820,0.038159,0.201376,0.495570,122.232394
4,ambient,0.367867,0.237162,-18.601365,0.041562,0.776790,0.676221,0.129283,0.167498,111.113129


## Define Widgets

In [7]:
# Defining widgets for "overview + details" panel

feature_picker = pnw.Select(name='Feature', options=audio_features)

def create_summary_plot(data, feature):
    genre_summary_melted = pd.melt(data, id_vars='track_genre', value_vars=audio_features, var_name='Audio Feature', value_name='Average Value')
    filtered_data = genre_summary_melted[genre_summary_melted['Audio Feature'] == feature]
    fig = px.bar(filtered_data, x='track_genre', y='Average Value',
                 title=f'Average {feature.capitalize()} per Genre',
                 color='Audio Feature',
                 width=1500)
    fig.update_layout(xaxis_tickangle=-45)

    return fig

plot1 = pn.bind(create_summary_plot,
                data=genre_summary,
                feature=feature_picker
                )

feature_summary_panel = pn.Column(
    pn.Row(feature_picker),
    plot1
)
feature_summary_panel

BokehModel(combine_events=True, render_bundle={'docs_json': {'6ceb7b03-219f-4810-978d-1d22e6385230': {'version…

In [8]:
# Defining widgets for scatter plot showing features vs. popularity by genre

x_axis_picker = pnw.Select(name='X-Axis', options=audio_features)
opacity_slider = pnw.FloatSlider(name='Opacity', start=0.1, end=1.0, step=0.1, value=0.5)
genre_picker = pnw.MultiChoice(name='Genres', options=genres)

def create_scatter_plot(data, x_axis, opacity, selected_genres):
    if selected_genres:
        data = data[data['track_genre'].isin(selected_genres)]

    fig = px.scatter(
        data,
        x=x_axis,
        y='popularity',
        color='popularity_bucket',
        opacity=opacity,
        title=f'Scatter Plot of {x_axis} vs Popularity <br>for {selected_genres} Genres',
        labels={x_axis: x_axis.capitalize(), 'popularity': 'Popularity'}
    )
    return fig

plot2 = pn.bind(create_scatter_plot,
                data=songs_pvf,
                x_axis=x_axis_picker,
                opacity=opacity_slider,
                selected_genres=genre_picker
                )

feature_v_popularity_panel = pn.Column(
    pn.Row(x_axis_picker, opacity_slider),
    pn.Row(plot2,genre_picker)
)
feature_v_popularity_panel

BokehModel(combine_events=True, render_bundle={'docs_json': {'841d8a9f-e2f1-47f0-bf8f-7e86a124ea75': {'version…